# Libraries


In [1]:
import sys
import os 
import time as time
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,  GradientBoostingClassifier
import tensorflow as tf
import keras
import re
import time
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
train_merge_extend = pd.read_csv('train_merge_extend.txt', sep = "|")
test_merge_extend  = pd.read_csv('test_merge_extend.txt', sep = "|")

In [3]:
import gc; gc.enable();

In [4]:
y_train = train_merge_extend.TARGET
train_merge_extend_tr = train_merge_extend.drop(['TARGET'], axis = 1)
del train_merge_extend


In [5]:
train_merge_extend_tr = train_merge_extend_tr.drop(['NAME_CONTRACT_TYPEcash_loan'], axis = 1)


In [6]:
train_merge_extend_tr = train_merge_extend_tr.drop(['NAME_CONTRACT_TYPEglobal'], axis = 1)

In [7]:
train_merge_extend_tr = train_merge_extend_tr.replace([np.inf, -np.inf], 0)

In [8]:
train_merge_extend_tr = train_merge_extend_tr.fillna(0)
test_merge_extend     = test_merge_extend.fillna(0)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_tr, y_val = train_test_split(train_merge_extend_tr, y_train ,  
                                                     random_state = 321, 
                                                     test_size = .3)

In [10]:
from sklearn.model_selection import StratifiedKFold, KFold

kfold = KFold(n_splits =10, 
                       random_state = 321).split(X_train, y_tr)

In [11]:
from sklearn.metrics import roc_auc_score

In [12]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, y_tr)
valid_data = lgb.Dataset(X_valid, y_val)




In [29]:
params = {'boosting_type': 'gbdt',
          'max_depth' : 7,
          'objective': 'binary',
          'nthread': 64,
          'num_leaves': 128,
          'learning_rate': 0.05,
          'max_bin': 64,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.4 ,
          'reg_alpha': 5,
          'reg_lambda': 5,
          'min_split_gain': 1.0,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'auc'
          }


In [30]:
lgbm = lgb.train(params,
                 train_data,
                 2500,
                 valid_sets=valid_data,
                 early_stopping_rounds= 100,
                 verbose_eval= 10
                 )

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.716781
[20]	valid_0's auc: 0.727321
[30]	valid_0's auc: 0.734124
[40]	valid_0's auc: 0.73834
[50]	valid_0's auc: 0.744552
[60]	valid_0's auc: 0.74861
[70]	valid_0's auc: 0.751219
[80]	valid_0's auc: 0.753529
[90]	valid_0's auc: 0.755089
[100]	valid_0's auc: 0.756594
[110]	valid_0's auc: 0.758518
[120]	valid_0's auc: 0.760087
[130]	valid_0's auc: 0.761614
[140]	valid_0's auc: 0.762914
[150]	valid_0's auc: 0.763802
[160]	valid_0's auc: 0.764706
[170]	valid_0's auc: 0.765327
[180]	valid_0's auc: 0.765861
[190]	valid_0's auc: 0.766487
[200]	valid_0's auc: 0.766796
[210]	valid_0's auc: 0.767221
[220]	valid_0's auc: 0.767658
[230]	valid_0's auc: 0.768061
[240]	valid_0's auc: 0.768468
[250]	valid_0's auc: 0.768725
[260]	valid_0's auc: 0.768967
[270]	valid_0's auc: 0.76926
[280]	valid_0's auc: 0.769509
[290]	valid_0's auc: 0.76979
[300]	valid_0's auc: 0.770065
[310]	valid_0's auc: 0.770241
[320]	valid_0's auc:

In [31]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [33]:
from xgboost import XGBClassifier

In [34]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [36]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_tr), verbose=3, random_state=1001 )

# Here we go
random_search.fit(X_train, y_tr)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=0.6, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=0.8 


In [20]:
scores = []

for k, (train, valid) in enumerate(kfold):
    print(max(train), min(train))
    print("Valid \n", max(valid), min(valid))
    rf.fit(X_train.iloc[train], y_tr.iloc[train] )
    pred = rf.predict(X_train.iloc[valid])
    score = roc_auc_score(pred , y_tr.iloc[valid])
    scores.append(score)
    print('Fold: %s,  Acc: %.3f' % (k+1, score))

215256 21526
Valid 
 21525 0
Fold: 1,  Acc: 0.603
215256 0
Valid 
 43051 21526


KeyboardInterrupt: 

In [21]:
from sklearn.ensemble import GradientBoostingClassifier

In [22]:
from sklearn.model_selection import StratifiedKFold, KFold

kfold = KFold(n_splits =10, 
                       random_state = 321).split(X_train, y_tr)

In [23]:
gbm = GradientBoostingClassifier()

scores = []

for k, (train, valid) in enumerate(kfold):
    print(max(train), min(train))
    print("Valid \n", max(valid), min(valid))
    gbm.fit(X_train.iloc[train], y_tr.iloc[train] )
    pred = gbm.predict(X_train.iloc[valid])
    score = roc_auc_score(pred , y_tr.iloc[valid])
    scores.append(score)
    print('Fold: %s,  Acc: %.3f' % (k+1, score))

215256 21526
Valid 
 21525 0


KeyboardInterrupt: 

In [24]:
import xgboost as xgboost

ModuleNotFoundError: No module named 'xgboost'

In [25]:
import lightgbm

In [26]:
!pip install xgboost


    100% |████████████████████████████████| 501kB 2.2MB/s eta 0:00:01
    Complete output from command python setup.py egg_info:
    + pwd
    + oldpath=/tmp/pip-build-rvkn8kiz/xgboost
    + cd ./xgboost/
    + echo
    + grep -q darwin
    + LIB_XGBOOST=libxgboost.so
    + make clean
    ./xgboost/build-python.sh: 37: ./xgboost/build-python.sh: make: not found
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-build-rvkn8kiz/xgboost/setup.py", line 42, in <module>
        LIB_PATH = libpath['find_lib_path']()
      File "/tmp/pip-build-rvkn8kiz/xgboost/xgboost/libpath.py", line 49, in find_lib_path
        'List of candidates:\n' + ('\n'.join(dll_path)))
    XGBoostLibraryNotFound: Cannot find XGBoost Library in the candidate path, did you install compilers and run build.sh in root path?
    List of candidates:
    /tmp/pip-build-rvkn8kiz/xgboost/xgboost/libxgboost.so
    /tmp/pip-build-rvkn8kiz/xgboost/xgboost/../../lib/libxgboost.s

In [ ]:
!conda install -c conda-forge xgboost

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [ ]:
!y